# Julia Workshop: Optimization and Solvers

Based on a notebook by Chase Coleman and Spencer Lyon and on material from QuantEcon

18 December 2017

## Goal

A few different packages used for optimization or non-linear equation solving.


## QuantEcon

The QuantEcon library has a few simple optimizers and solvers. See the economics [notebook](economics.ipynb) for more information.

In [1]:
# Pkg.add("QuantEcon")

## Optim.jl

Package for optimizing written in pure Julia

[Documentation](http://julianlsolvers.github.io/Optim.jl/stable/)

In [2]:
# Pkg.add("Optim")

In [1]:
using Optim

In [2]:
rosenbrock(x::Vector) = (1.0 - x[1])^2 + 100.0*(x[2] - x[1]^2)^2

rosenbrock (generic function with 1 method)

### Optimizing without gradient

In [9]:
optimize(rosenbrock, zeros(2), NelderMead())

Results of Optimization Algorithm
 * Algorithm: Nelder-Mead
 * Starting Point: [0.0,0.0]
 * Minimizer: [0.9999634355313174,0.9999315506115275]
 * Minimum: 3.525527e-09
 * Iterations: 60
 * Convergence: true
   *  √(Σ(yᵢ-ȳ)²)/n < 1.0e-08: true
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 117

In [10]:
optimize(rosenbrock, zeros(2), BFGS())

Results of Optimization Algorithm
 * Algorithm: BFGS
 * Starting Point: [0.0,0.0]
 * Minimizer: [0.9999999926033423,0.9999999852005353]
 * Minimum: 5.471433e-17
 * Iterations: 16
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 3.47e-07 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = 1.20e+03 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 2.33e-09 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 53
 * Gradient Calls: 53

### Optimizing with gradient

In [9]:
function rosenbrock_grad!(grad::Vector, x::Vector)
    grad[1] = -2.0*(1.0 - x[1]) - 400.0*(x[2] - x[1]^2)*x[1]
    grad[2] = 200.0*(x[2] - x[1]^2)
end

rosenbrock_grad! (generic function with 1 method)

In [11]:
optimize(rosenbrock, rosenbrock_grad!, .5 * ones(2), LBFGS(),
         Optim.Options(x_tol=1e-10, f_tol=1e-9, iterations=25000,
                       allow_f_increases=true))

Results of Optimization Algorithm
 * Algorithm: L-BFGS
 * Starting Point: [0.5,0.5]
 * Minimizer: [1.000000000000618,1.0000000000011904]
 * Minimum: 5.890616e-25
 * Iterations: 15
 * Convergence: true
   * |x - x'| < 1.0e-10: false 
     |x - x'| = 7.01e-09 
   * |f(x) - f(x')| / |f(x)| < 1.0e-09: false
     |f(x) - f(x')| / |f(x)| = 3.07e-08 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 1.94e-11 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 46
 * Gradient Calls: 46

In [12]:
optimize(rosenbrock, rosenbrock_grad!, zeros(2), GradientDescent(),
         Optim.Options(x_tol=1e-10, f_tol=1e-9, iterations=25000,
                       allow_f_increases=true))

Results of Optimization Algorithm
 * Algorithm: Gradient Descent
 * Starting Point: [0.0,0.0]
 * Minimizer: [0.9999999572646519,0.9999999142802773]
 * Minimum: 1.832511e-15
 * Iterations: 18460
 * Convergence: true
   * |x - x'| < 1.0e-10: true 
     |x - x'| = 9.99e-11 
   * |f(x) - f(x')| / |f(x)| < 1.0e-09: false
     |f(x) - f(x')| / |f(x)| = 2.95e-09 
   * |g(x)| < 1.0e-08: false 
     |g(x)| = 4.98e-08 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 46182
 * Gradient Calls: 46182

### Optimizing with Hessian

In [13]:
function rosenbrock_hess!(hess::Matrix, x::Vector)
    hess[1, 1] = 2.0 - 400.0 * x[2] + 1200.0*x[1]^2
    hess[1, 2] = -400.0 * x[1]
    hess[2, 1] = -400.0 * x[1]
    hess[2, 2] = 200.0
end

rosenbrock_hess! (generic function with 2 methods)

In [14]:
optimize(rosenbrock, rosenbrock_grad!, rosenbrock_hess!, zeros(2), Newton())

Results of Optimization Algorithm
 * Algorithm: Newton's Method
 * Starting Point: [0.0,0.0]
 * Minimizer: [0.9999999999999994,0.9999999999999989]
 * Minimum: 3.081488e-31
 * Iterations: 14
 * Convergence: true
   * |x - x'| < 1.0e-32: false 
     |x - x'| = 3.06e-09 
   * |f(x) - f(x')| / |f(x)| < 1.0e-32: false
     |f(x) - f(x')| / |f(x)| = 2.94e+13 
   * |g(x)| < 1.0e-08: true 
     |g(x)| = 1.11e-15 
   * Stopped by an increasing objective: false
   * Reached Maximum Number of Iterations: false
 * Objective Calls: 44
 * Gradient Calls: 44
 * Hessian Calls: 14

## NLopt

Julia wrapper to high quality C library.

This library has _lots_ of options for algorithms. See [list](http://ab-initio.mit.edu/wiki/index.php/NLopt_Algorithms) of algorithms

[Julia package](https://github.com/JuliaOpt/NLopt.jl) and [C Documentation](http://ab-initio.mit.edu/wiki/index.php/NLopt)

Note that functions optimized with NLopt have a different ordering of arguments compared to our previous examples using Optim.jl

In [12]:
# Pkg.add("NLopt")

In [15]:
using NLopt

In [20]:
function rosenbrock_nlopt(x::Vector, grad::Vector)
    if length(grad) > 0
        rosenbrock_grad!(grad, x)
    end

    return rosenbrock(x)
end

rosenbrock_nlopt (generic function with 1 method)

### A non-gradient and gradient based method

In [18]:
opt_LBFGS = Opt(:LD_LBFGS, 2)

min_objective!(opt_LBFGS, rosenbrock_nlopt)
xtol_rel!(opt_LBFGS, 1e-10)
ftol_rel!(opt_LBFGS, 1e-9)

NLopt.optimize(opt_LBFGS, zeros(2))

(3.251384063527492e-21, [1.0, 1.0], :SUCCESS)

In [19]:
opt_PRAXIS = Opt(:LN_PRAXIS, 2)

min_objective!(opt_PRAXIS, rosenbrock_nlopt)

NLopt.optimize(opt_PRAXIS, zeros(2))

(1.232595164407831e-30, [1.0, 1.0], :SUCCESS)

### A constrained optimization problem

\begin{align*}
  \min_{x_1, x_2} &\sqrt{x_2} \\
  &\text{subject to } \\
  &x_2 \geq 0 \\
  &x_2 \geq (2 x_1)^3  \\
  &x_2 \geq (-x_1 + 1)^3
\end{align*}

In [21]:
function myfunc(x::Vector, grad::Vector)
    if length(grad) > 0
        grad[1] = 0
        grad[2] = 0.5/sqrt(x[2])
    end

    return sqrt(x[1] + x[2])
end

function myconstraint(x::Vector, grad::Vector, a, b)
    if length(grad) > 0
        grad[1] = 3a * (a*x[1] + b)^2
        grad[2] = -1
    end

    return (a*x[1] + b)^3 - x[2]
end

myconstraint (generic function with 1 method)

In [22]:
opt = Opt(:LD_MMA, 2)

lower_bounds!(opt, [-Inf, 0.])
xtol_rel!(opt, 1e-6)

min_objective!(opt, myfunc)

inequality_constraint!(opt, (x,g) -> myconstraint(x, g, 2.0, 0.0), 1e-8)
inequality_constraint!(opt, (x,g) -> myconstraint(x, g, -1.0 ,1.0), 1e-8)

(minf, minx, ret) = NLopt.optimize(opt, [1.234, 5.678])

println("got $minf at $minx (returned $ret)")

got 0.7934920514599207 at [0.333333, 0.296296] (returned XTOL_REACHED)


## NLsolve

Julia package written to solve systems of non-linear equations

[Documentation](https://github.com/JuliaNLSolvers/NLsolve.jl)

In [19]:
# Pkg.add("NLsolve")

In [23]:
using NLsolve

In [24]:
function f!(xy::Vector, fxy::Vector)
    # Pull out arguments
    x, y = xy

    # Fill fxy
    fxy[1] = x^2 - sin(y)
    fxy[2] = y^2 - cos(x)
end

function g!(xy::Vector, jacxy::Matrix)
    x, y = xy
    # Fill with derivatives of first function
    jacxy[1, 1] = 2*x
    jacxy[1, 2] = -cos(y)

    # Fill off-diagonal
    jacxy[2, 1] = sin(x)
    jacxy[2, 2] = 2*y
end

g! (generic function with 1 method)

In [25]:
res = nlsolve(f!, g!, [0.5, 0.5], ftol=1e-8)

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.5, 0.5]
 * Zero: [0.8517, 0.811606]
 * Inf-norm of residuals: 0.000000
 * Iterations: 5
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 6
 * Jacobian Calls (df/dx): 6